In [19]:
# Imports required for handling the diffusion model
from diffusers import StableDiffusionPipeline
from diffusers.models import UNet2DConditionModel
from diffusers.models.embeddings import TimestepEmbedding
from diffusers import DDPMScheduler

# from transformers import CLIPTextModel
import torch

import torch.nn.functional as F
from torch.utils.data import DataLoader

from torch import nn
import torchvision
# from torchsummary import summary


from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

In [20]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cpu
False


In [21]:
torch.cuda.is_available()

False

In [22]:


# Define the projection layer
# This is a small network to embed the 2D vector into a higher dimension
projector = torch.nn.Sequential(
    torch.nn.Linear(2, 128),
    torch.nn.SiLU(),
    torch.nn.Linear(128, 512) # Dimension must match the expected embedding dimension in the Diffusion model
)

class MyDiffusionModel(StableDiffusionPipeline):
    def __init__(self, *args, **kwargs):
        super().__init__(**kwargs)
        # Replace the original text encoder with our custom projection layer
        self.unet = UNet2DConditionModel(
            # ... other UNet parameters
            class_embed_type='class_embedding', # Crucial for 2D conditioning
            num_class_embeds=512, # Dimension of our projected embedding
            text_encoders=None,
            # Add our projector after a copy
            add_encoder_output=True,
            encoders=[torch.nn.Identity(copy=True), projector],
        )


# Create the model instance
model = MyDiffusionModel()


# Generate images with a 2D vector condition
vector = torch.tensor([0.5, 0.7]) # Adjust values to mean something (e.g., brightness, contrast)
print(model.generate(prompt="A photorealistic image", vector=vector))

ImportError: 
MyDiffusionModel requires the transformers library but it was not found in your environment. You can install it with pip: `pip
install transformers`


In [ ]:
from diffusers import UNet2DConditionModel


projector = torch.nn.Sequential(
    torch.nn.Linear(2, 128),
    torch.nn.SiLU(),
    torch.nn.Linear(128, 512) # Dimension must match the expected embedding dimension in the Diffusion model
)

# model = UNet2DConditionModel(
#     sample_size=config.image_size,  # the target image resolution
#     in_channels=3,  # the number of input channels, 3 for RGB images
#     out_channels=3,  # the number of output channels
#     layers_per_block=2,  # how many ResNet layers to use per UNet block
#     block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channes for each UNet block
#     down_block_types=( 
#         "DownBlock2D",  # a regular ResNet downsampling block
#         "DownBlock2D", 
#         "DownBlock2D", 
#         "DownBlock2D", 
#         "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
#         "DownBlock2D",
#     ), 
#     up_block_types=(
#         "UpBlock2D",  # a regular ResNet upsampling block
#         "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
#         "UpBlock2D", 
#         "UpBlock2D", 
#         "UpBlock2D", 
#         "UpBlock2D"  
#       ),
# )


unet = UNet2DConditionModel(
    # ... other UNet parameters
    class_embed_type='class_embedding', # Crucial for 2D conditioning
    num_class_embeds=512, # Dimension of our projected embedding
    text_encoders=None,
    # Add our projector after a copy
    add_encoder_output=True,
    encoders=[torch.nn.Identity(copy=True), projector],
        )

In [ ]:
import torch
from torch import nn
from diffusers import UNet2DConditionModel
from diffusers.models.embeddings import TimestepEmbedding

# Let's assume your conditioning vector is a 2D vector: [x, y]
class CustomConditionedUNet(nn.Module):
    def __init__(self, base_unet: UNet2DConditionModel, cond_dim=2, embedding_dim=128):
        super().__init__()
        self.unet = base_unet
        self.cond_embed = nn.Sequential(
            nn.Linear(cond_dim, embedding_dim),
            nn.SiLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )
        self.unet.add_embedding("custom_conditioning", embedding_dim)


    def forward(self, sample, timestep, encoder_hidden_states=None, cond_vector=None):
        # Embed the custom conditioning vector
        if cond_vector is not None:
            cond_emb = self.cond_embed(cond_vector)
        else:
            cond_emb = None

        # Pass it into the model as an extra embedding
        return self.unet(
            sample=sample,
            timestep=timestep,
            encoder_hidden_states=encoder_hidden_states,
            added_cond_kwargs={"custom_conditioning": cond_emb}
        )

In [3]:
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as T
import pandas as pd
# import torch
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

class ImageVectorDataset(Dataset):
    def __init__(self, image_dir, label_csv, image_size=64):
        self.df = pd.read_csv(label_csv)
        self.image_dir = image_dir
        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor(), 
            T.Normalize([0.5]*3, [0.5]*3)
            
        ])

    def __len__(self):
        return len(self.df)


    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(os.path.join(self.image_dir, row['filename'])).convert("RGB")
        # print("image ", image.size)
        # image.show(title="before")
        image = self.transform(image)

        # print(torch.min(image), torch.max(image))

        # image = image.clamp(0, 1) # -1 to 1
        # back_to_pil = T.ToPILImage()(image)
        # back_to_pil.show(title="after")

        vector = torch.tensor([row['x'], row['y']], dtype=torch.float32)
        return image, vector

In [4]:
dataset = ImageVectorDataset("data/images", "data/labels.csv", image_size=224)

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
print(len(dataloader))

# for images, cond_vectors in tqdm(dataloader):
#     images = images.cuda()
#     cond_vectors = cond_vectors.cuda()

120


In [70]:
for i, (x,y) in enumerate(dataloader):
    if i < 1:
        pass
        # print(x[0].shape)  
        # print(x[0].permute(1,2,0).shape)
        # print(torch.max(x[0]))
        # print(torch.min(x[0]))

        # PIL_image = x[0].permute(1,2,0).cpu().numpy()
        # # torchvision.transforms.ToPILImage(PIL_image)
        # print(type(PIL_image), PIL_image.size)
        # PIL_image2 = Image.fromarray(PIL_image, 'RGB')

        # PIL_image2.show()
        # plt.imshow(x[0].permute(1,2,0))
        # plt.show()
    else:
        break

In [71]:
x[0].shape

torch.Size([3, 224, 224])

In [57]:
class CustomConditionedUNet(nn.Module):
    def __init__(self, base_unet: UNet2DConditionModel, cond_dim=2, embedding_dim=128):
        super().__init__()
        self.unet = base_unet
        self.cond_embed = nn.Sequential(
            nn.Linear(cond_dim, embedding_dim),
            nn.SiLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )


    def forward(self, sample, timestep, cond_vector=None):
        # [B, 128] -> [B, 1, 128] (cross-attention format)
        cond_emb = self.cond_embed(cond_vector).unsqueeze(1)
        return self.unet(
            sample=sample,
            timestep=timestep,
            encoder_hidden_states=cond_emb
        )

In [ ]:



# # --- Dataset ---
# dataset = ImageVectorDataset("data/images", "data/labels.csv", image_size=64)
# dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# # # --- Model & Noise Scheduler ---
# # from diffusers import UNet2DModel  # vanilla UNet for image denoising
# base_unet = UNet2DConditionModel(
#     sample_size=64, in_channels=3, out_channels=3, layers_per_block=2,
#     block_out_channels=(64, 128, 256, 256),  # now 4 blocks = 4 values
#     down_block_types=('DownBlock2D', 'AttnDownBlock2D', 'AttnDownBlock2D', 'AttnDownBlock2D'),
#     up_block_types=('AttnUpBlock2D', 'AttnUpBlock2D', 'AttnUpBlock2D', 'UpBlock2D'),
#     cross_attention_dim=128
# )


# model = CustomConditionedUNet(base_unet).cuda()

# noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# # --- Training ---
# for epoch in range(200):
#     for images, cond_vectors in tqdm(dataloader):
#         images = images.cuda()
#         cond_vectors = cond_vectors.cuda()


#         # Add noise to images
#         noise = torch.randn_like(images)
#         timesteps = torch.randint(0, 1000, (images.size(0),), device=images.device).long()
#         noisy_images = noise_scheduler.add_noise(images, noise, timesteps)

#         # Predict the noise
#         pred = model(noisy_images, timestep=timesteps, cond_vector=cond_vectors).sample

#         loss = F.mse_loss(pred, noise)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


# torch.save(model.state_dict(), "cond_unet.pt")

100%|██████████| 30/30 [00:02<00:00, 11.98it/s]


Epoch 0: Loss = 0.1412


100%|██████████| 30/30 [00:02<00:00, 13.85it/s]


Epoch 1: Loss = 0.1243


100%|██████████| 30/30 [00:02<00:00, 13.73it/s]


...

Epoch 197: Loss = 0.0066


100%|██████████| 30/30 [00:02<00:00, 13.67it/s]


Epoch 198: Loss = 0.0018


100%|██████████| 30/30 [00:02<00:00, 13.64it/s]


Epoch 199: Loss = 0.0015


In [ ]:

# https://huggingface.co/docs/diffusers/en/tutorials/basic_training


from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
import os


from diffusers import DDPMPipeline
from diffusers.utils import make_image_grid
import os

def evaluate(config, epoch, pipeline):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.Generator(device='cpu').manual_seed(config.seed), # Use a separate torch generator to avoid rewinding the random state of the main training loop
    ).images

    # Make a grid out of the images
    image_grid = make_image_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")




def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging

    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True
            ).repo_id
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    epoch_min_loss = 1000
    epoch_loss = []


    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, (clean_images, cond_vectors) in enumerate(train_dataloader):
            # clean_images = batch["images"]
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device,
                dtype=torch.int64
            )

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)

            
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                # Predict the noise residual
                # noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                noise_pred = model(noisy_images, timestep=timesteps, cond_vector=cond_vectors).sample

                # print(noise_pred.size())
                # print(torch.min(noise_pred), torch.max(noise_pred))

                # noise_pred = noise_pred[0].clamp(-1, 1) # -1 to 1
                # back_to_pil = T.ToPILImage()(noise_pred)
                # back_to_pil.show(title="after")
                        

                loss = F.mse_loss(noise_pred, noise)
                epoch_loss.append(loss.detach().item())
                # if loss.detach().item() < min_loss:
                #     min_loss = loss.detach().item()

                #     if (epoch+1) > 100:
                #         torch.save(model.state_dict(), f"{config.output_dir}_cond_unet_min_loss.pt")
                #         print("Saved ", min_loss.3f)

                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        mean_epoch_loss = sum(epoch_loss)/len(epoch_loss)
        if mean_epoch_loss < epoch_min_loss:
            epoch_min_loss = mean_epoch_loss

            if (epoch+1) > 100:
                torch.save(model.state_dict(), f"{config.output_dir}_cond_unet_min_loss.pt")
                print("Saved ", epoch_min_loss)

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                # evaluate(config, epoch, pipeline)
                pass

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                if config.push_to_hub:
                    upload_folder(
                        repo_id=repo_id,
                        folder_path=config.output_dir,
                        commit_message=f"Epoch {epoch}",
                        ignore_patterns=["step_*", "epoch_*"],
                    )
                else:
                    pass
                    #pipeline.save_pretrained(config.output_dir)
                    # torch.save(model.state_dict(), f"{config.output_dir}_cond_unet_{epoch + 1}.pt")

                    

In [102]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 64  # the generated image resolution
    train_batch_size = 8
    eval_batch_size = 8  # how many images to sample during evaluation
    num_epochs = 500
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 1
    save_model_epochs = 50
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "ddpm-test-128"  # the model name locally and on the HF Hub

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    hub_model_id = "<your-username>/<my-awesome-model>"  # the name of the repository to create on the HF Hub
    hub_private_repo = None
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0


config = TrainingConfig()



# --- Dataset ---
dataset = ImageVectorDataset("data/images", "data/labels.csv", image_size=64)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# # --- Model & Noise Scheduler ---
# from diffusers import UNet2DModel  # vanilla UNet for image denoising
base_unet = UNet2DConditionModel(
    sample_size=64, in_channels=3, out_channels=3, layers_per_block=2,
    block_out_channels=(64, 128, 256),  # now 4 blocks = 4 values
    down_block_types=('DownBlock2D', 'AttnDownBlock2D',  'AttnDownBlock2D'),
    up_block_types=('AttnUpBlock2D', 'AttnUpBlock2D',  'UpBlock2D'),
    cross_attention_dim=128
)

model = CustomConditionedUNet(base_unet).cuda()


from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(dataloader) * config.num_epochs),
)

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

In [103]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)



c:\Users\SANO\.conda\envs\mm_diff_3\Lib\site-packages\accelerate\accelerator.py:506: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")


Launching training on one GPU.


Epoch 2:  13%|█▎        | 2/15 [00:00<00:00, 15.21it/s, loss=1.04, lr=6.4e-6, step=31]






Epoch 48: 100%|██████████| 15/15 [00:01<00:00, 14.96it/s, loss=0.0232, lr=9.97e-5, step=734]
self.unet=CustomConditionedUNet(
  (unet): UNet2DConditionModel(
    (conv_in): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (time_proj): Timesteps()
    (time_embedding): TimestepEmbedding(
      (linear_1): Linear(in_features=64, out_features=256, bias=True)
      (act): SiLU()
      (linear_2): Linear(in_features=256, out_features=256, bias=True)
    )
    (down_blocks): ModuleList(
      (0): DownBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 64, eps=1e-05, affine=True)
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (time_emb_proj): Linear(in_features=256, out_features=64, bias=True)
            (norm2): GroupNorm(32, 64, eps=1e-05, affine=True)
            (dro

Saved  0.08959714172933583


Epoch 100: 100%|██████████| 15/15 [00:01<00:00,  9.95it/s, loss=0.0051, lr=9.49e-5, step=1514]


...

Saved  0.02089807834520492


Epoch 498: 100%|██████████| 15/15 [00:00<00:00, 15.28it/s, loss=0.00115, lr=1.13e-9, step=7484]

Saved  0.020859226239154597


Epoch 498: 100%|██████████| 15/15 [00:01<00:00, 14.11it/s, loss=0.00115, lr=1.13e-9, step=7484]
self.unet=CustomConditionedUNet(
  (unet): UNet2DConditionModel(
    (conv_in): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (time_proj): Timesteps()
    (time_embedding): TimestepEmbedding(
      (linear_1): Linear(in_features=64, out_features=256, bias=True)
      (act): SiLU()
      (linear_2): Linear(in_features=256, out_features=256, bias=True)
    )
    (down_blocks): ModuleList(
      (0): DownBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 64, eps=1e-05, affine=True)
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (time_emb_proj): Linear(in_features=256, out_features=64, bias=True)
            (norm2): GroupNorm(32, 64, eps=1e-05, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3

Saved  0.020820150084001943


In [84]:
def generate_image(model, scheduler, x, y, steps=50, size=64):
    model.eval().cuda()
    scheduler.set_timesteps(steps)

    cond_vector = torch.tensor([[x, y]], device='cuda')
    latents = torch.randn((1, 3, size, size), device='cuda')

    with torch.no_grad():
        for t in scheduler.timesteps:
            noise_pred = model(latents, t, cond_vector=cond_vector).sample
            latents = scheduler.step(noise_pred, t, latents).prev_sample

            print(latents.shape)

    img = (latents[0].clamp(-1, 1) + 1) / 2
    img = img.cpu().permute(1, 2, 0).numpy()
    return img

In [106]:
scheduler = DDPMScheduler(num_train_timesteps=100)


img = generate_image(model, scheduler, x=12., y=0.)
# plt.imshow(img)
# plt.axis('off')
# plt.show()

torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
torch.Size([1, 3, 64, 64])
t

In [86]:
print(img.shape)
print(type(img))
print(np.min(img))
print(np.max(img))
print(np.unique(img))

(64, 64, 3)
<class 'numpy.ndarray'>
0.29904577
0.7401182
[0.29904577 0.31451488 0.3286583  ... 0.72841144 0.73898107 0.7401182 ]


In [17]:
# plt.imshow(img)

In [87]:
type(img)

numpy.ndarray

In [107]:
PIL_image = Image.fromarray(img, 'RGB')
PIL_image.size
PIL_image.show()

In [43]:
def generate_image2(model, scheduler, x, y, steps=50, size=64):
    model.eval().cuda()
    scheduler.set_timesteps(steps)

    cond_vector = torch.tensor([[x, y]], device='cuda')
    latents = torch.randn((1, 3, size, size), device='cuda')
    i = 0

    with torch.no_grad():
        for t in scheduler.timesteps:
            noise_pred = model(latents, t, cond_vector=cond_vector).sample
            latents = scheduler.step(noise_pred, t, latents).prev_sample

            print(t, i)
            i +=1 

            if i > 40:
                img = (latents[0].clamp(-1, 1) + 1) / 2
                img = img.cpu().permute(1, 2, 0).numpy()

                PIL_image = Image.fromarray(img, 'RGB')
                PIL_image.show()
            # return img

            

In [105]:

scheduler = DDPMScheduler(num_train_timesteps=100)

img = generate_image2(model, scheduler, x=60., y=0.)


tensor(98) 0
tensor(96) 1
tensor(94) 2
tensor(92) 3
tensor(90) 4
tensor(88) 5
tensor(86) 6
tensor(84) 7
tensor(82) 8
tensor(80) 9
tensor(78) 10
tensor(76) 11
tensor(74) 12
tensor(72) 13
tensor(70) 14
tensor(68) 15
tensor(66) 16
tensor(64) 17
tensor(62) 18
tensor(60) 19
tensor(58) 20
tensor(56) 21
tensor(54) 22
tensor(52) 23
tensor(50) 24
tensor(48) 25
tensor(46) 26
tensor(44) 27
tensor(42) 28
tensor(40) 29
tensor(38) 30
tensor(36) 31
tensor(34) 32
tensor(32) 33
tensor(30) 34
tensor(28) 35
tensor(26) 36
tensor(24) 37
tensor(22) 38
tensor(20) 39
tensor(18) 40
tensor(16) 41
tensor(14) 42
tensor(12) 43
tensor(10) 44
tensor(8) 45
tensor(6) 46
tensor(4) 47
tensor(2) 48
tensor(0) 49


Load model

In [104]:
base_unet = UNet2DConditionModel(
    sample_size=64, in_channels=3, out_channels=3, layers_per_block=2,
    block_out_channels=(64, 128, 256),  # now 4 blocks = 4 values
    down_block_types=('DownBlock2D', 'AttnDownBlock2D',  'AttnDownBlock2D'),
    up_block_types=('AttnUpBlock2D', 'AttnUpBlock2D',  'UpBlock2D'),
    cross_attention_dim=128
)


model = CustomConditionedUNet(base_unet).cuda()

model.load_state_dict(torch.load("ddpm-test-128_cond_unet_min_loss.pt", weights_only=True))
# model.eval()

<All keys matched successfully>

In [49]:
angles = range(0,360,3)
len(angles)

120

In [51]:
import numpy as np
from PIL import Image

def simulate_horizontal_orbit_gradient_sphere(size=224, radius=80, camera_angle_deg=0):
    """
    Simulate a camera orbiting around the vertical (Z) axis of a gradient-colored sphere.

    camera_angle_deg: horizontal orbit angle in degrees (0 = front, 90 = side)
    """
    #camera_angle_deg: angle in degrees (0 = bottom-to-top gradient, 90 = left-to-right)

    image = np.zeros((size, size, 3), dtype=np.uint8)

    angle_rad = np.radians(camera_angle_deg)

    # Center of image
    center_x, center_y = size // 2, size // 2

    # Rotate the gradient direction vector (default is vertical: (0, -1))
    grad_dx = np.sin(angle_rad)
    grad_dy = -np.cos(angle_rad)  # negative to go from bottom to top by default

    for y in range(size):
        for x in range(size):
            dx = x - center_x
            dy = y - center_y
            distance = np.sqrt(dx**2 + dy**2)

            if distance <= radius:
                # Compute dot product between (dx, dy) and gradient direction
                dot = dx * grad_dx + dy * grad_dy

                # Normalize to [0, 1] across the circle
                t = (dot + radius) / (2 * radius)
                t = np.clip(t, 0, 1)

                r = int(255 * (1 - t))  # Red fades out
                g = 0
                b = int(255 * t)        # Blue fades in

                image[y, x] = (r, g, b)

    return image

# Example: simulate camera rotating around the circle in image plane
# angles = [0, 45, 90, 135, 180]  # Camera rotating from front to back
# angles = [5,10,15,20]
angles = range(0,360,3)

for angle in angles:
    img = simulate_horizontal_orbit_gradient_sphere(camera_angle_deg=angle)
    # plt.imshow(img)
    # plt.show()
    # images_dir = '/content/gdrive/My Drive/Images'
    # plt.savefig(f"{images_dir}/label{unique_arr[i]}.png")

    Image.fromarray(img).save(f"data/images/gradient_circle_{angle}.png")

In [4]:
import csv
import random

def generate_random_csv(filename='labels.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['filename', 'x', 'y'])  # header
        angles = range(0,360,3)

        for i in angles:
            fname = f"gradient_circle_{i}.png"
            x = i
            y = 0
            writer.writerow([fname, x, y])

    print(f"CSV file '{filename}' with rows created.")

# Example usage
generate_random_csv("labels.csv")

CSV file 'labels.csv' with rows created.
